<a href="https://colab.research.google.com/github/RautRitesh/langgraph/blob/main/Human_in_the_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langchain-groq langgraph langchain-community langchain-tavily

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


##Human in the loop👱➿➿ can be implemented using langgraph inbuilt classes as well as by the custom logic. For today I will be going with custom logic

##Step1: Importing all the required packages and the tools


In [9]:
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langchain_core.tools import tool
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage

In [10]:
from google.colab import userdata
tavily_api_key=userdata.get('Tavily_api_key')
api_key=userdata.get('groq_api_key')

In [11]:
llm=ChatGroq(model="openai/gpt-oss-20b", api_key=api_key)

##Step2:Setting up the tools

In [12]:
@tool
def multiply(a,b):
  """This is a tool for multiplying two numbers"""
  return a*b



In [13]:
search_tool=TavilySearch(max_results=2,tavily_api_key=tavily_api_key)

In [15]:
@tool
def search(query:str):
  """This  is a tavily search tool used to search the internet for information"""
  response=search_tool.invoke(query)
  return response

In [18]:
tools=[multiply, search]

In [23]:
llm_with_tools=llm.bind_tools(tools=tools)

### Custom logic for invoking the tools by ourself and understanding how the llm_with_tools given response👏👏👏⚒️⚒️


In [43]:
response=[llm_with_tools.invoke("who is pm of Koshi province nepal?")]

In [44]:
print(response[-1])

content='' additional_kwargs={'reasoning_content': 'User asks: "who is pm of Koshi province nepal?" They want the name of the provincial government head. Let\'s recall: Nepal\'s provinces: Koshi Province (Province No. 1). The provincial government has a Chief Minister. The current Chief Minister? As of my knowledge cutoff 2024-06: The Chief Minister of Province No. 1 (Koshi Province) is Rajendra Kumar Rai? Wait, let\'s recall: In 2023, the Chief Minister of Province No. 1 (Koshi Province) is Rajendra Kumar Rai of CPN (UML). Actually, I recall that in 2023, the Chief Minister was Rajendra Kumar Rai (Rajan). But let\'s verify. Let\'s search.', 'tool_calls': [{'id': 'fc_f21df68e-9951-4267-b529-584afba37121', 'function': {'arguments': '{"query":"Chief Minister of Koshi Province Nepal 2024"}', 'name': 'search'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 182, 'prompt_tokens': 165, 'total_tokens': 347, 'completion_time': 0.187497514, 'completion_tokens_deta

In [45]:
tool_mapping={tool.name:tool for tool in tools}

In [46]:
print(tool_mapping)

{'multiply': StructuredTool(name='multiply', description='This is a tool for multiplying two numbers', args_schema=<class 'langchain_core.utils.pydantic.multiply'>, func=<function multiply at 0x7dec07d5b600>), 'search': StructuredTool(name='search', description='This  is a tavily search tool used to search the internet for information', args_schema=<class 'langchain_core.utils.pydantic.search'>, func=<function search at 0x7dec07d5b880>)}


In [47]:
tool_details=response[-1].additional_kwargs.get("tool_calls",[])
if len(tool_details):
  function=tool_details[0]
  print(function)
  name=tool_details[0].get('function').get('name')
  query=tool_details[0].get('function').get('arguments')
  response=tool_mapping[name].invoke(query)

  print("\n--")
  print(name)
  print(query)
  print("final response from tools---")
  print(response)
else:
  print("no tool calls")

{'id': 'fc_f21df68e-9951-4267-b529-584afba37121', 'function': {'arguments': '{"query":"Chief Minister of Koshi Province Nepal 2024"}', 'name': 'search'}, 'type': 'function'}

--
search
{"query":"Chief Minister of Koshi Province Nepal 2024"}
final response from tools---
{'query': '{query:Chief Minister of Koshi Province Nepal 2024}', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://grokipedia.com/page/office_of_the_chief_minister_and_council_of_ministers_koshi_province', 'title': 'Office of the Chief Minister and Council of Ministers (Koshi Province)', 'content': 'As of late 2024, Hikmat Kumar Karki of the CPN-UML party serves as Chief Minister, having expanded the cabinet to include ministers from allied parties amid', 'score': 0.999966, 'raw_content': None}, {'url': 'https://radionepalonline.com/en/2024/05/09/378055.html', 'title': 'Karki appointed Koshi province CM', 'content': 'Biratnagar, May 9: Hikmat Bahadur Karki has been appointed as the C

## Step3: Now based on the `response given by llm` and `its understanding of how output is produced` now let us go with the creation of nodes.Remember that the `tool_mapping`🛠️:🗺️ was used because it store the information about our tools especially `tool object` that can  be invoked otherwise we could not do so , Hence the tool_mapping is important